# **Handwritten  Digit Prediction**

**Dataset:**

The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine provided by MNIST. The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

**Target:**

Goal is to correctly identify digits from a dataset of tens of thousands of handwritten images.

**Approach:**

Applied Data Augmentation for better prediction and large dataset, followed by model training using Convolutional Neural Network.

# **Importing Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#convolutional layers
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D
#core layers
from keras.layers import Dense, Dropout, Flatten

from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Data Preparation**

**Loading Data**

In [ ]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

**Splitting data into features and labels**

In [ ]:
X=train.drop(['label'],1).values
y=train['label'].values
test_x=test.values

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
print(test_x)

**Normalization**

Performing grayscale normalization to reduce the effect of illumination's differences.

Moreover the CNN converg faster on [0..1] data than on [0..255]

In [ ]:
X=X/255
test_x=test_x/255

**Reshaping Data**

In [ ]:
# Reshaping image into 3 dimensions (height = 28px, width = 28px , canal = 1)
# canal = 1 => For gray scale
X=X.reshape(-1,28,28,1)
test_x=test_x.reshape(-1,28,28,1)

**Label Encoding**

One-Hot Encoding

Labels are 10 digits numbers from 0 to 9. We need to encode these lables to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0]).

In [ ]:
y=to_categorical(y)
print(y)

**Splitting data into train and validation set**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=0)

Splitting the train set into two parts : a small fraction (10%) is the validation set used to evaluate the model and the rest (90%) is used to train the model.

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
print(X_train)

# **Data Visualisation**

In [ ]:
X_train__ = X_train.reshape(X_train.shape[0],28,28)

In [ ]:
fig,axis=plt.subplots(1,4,figsize=(20,10))
for i,ax in enumerate(axis.flat):
  ax.imshow(X_train__[i], cmap='binary')
  digit=y_train[i].argmax()
  ax.set(title=f"Real Number is {digit}");

Standardization

In [ ]:
#creating mean=0 and standard deviation=1
mean=np.mean(X_train)
std=np.std(X_train)
def standardize(x):
  return ((x-mean)/std)

In [ ]:
epochs=50 #for no of passes in training to optimize error
batch_size=64

# **Defining the model**

**CNN**

In [ ]:
model=Sequential()

#model.add(Lambda(standardize,input_shape=(28,28,1)))    
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
    
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# **Data Augumentation**

In [ ]:
#Data augmentation to prevent overfitting
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#datagen.fit(X_train)
train_gen=datagen.flow(X_train, y_train, batch_size=batch_size)
test_gen=datagen.flow(X_test, y_test, batch_size=batch_size)
 

# **Model Training**

In [ ]:
#This make accuarcy 0.998
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback() 
model.fit(X,y,batch_size=batch_size, validation_split=0.2, epochs=10,callbacks=[callbacks])

In [ ]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()      
# Fit the model
history = model.fit_generator(train_gen, 
                              epochs = epochs, 
                              steps_per_epoch = X_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks=[callbacks],
                              )

**Plotting CNN model**

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


# **Evaluating the model**

**Training and validation curves**

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1, figsize=(18, 10))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)


# **Confusion Matrix**

In [ ]:
fig = plt.figure(figsize=(10, 10)) # Set Figure

y_pred = model.predict(X_test) # Predict encoded label as 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

Y_pred = np.argmax(y_pred, 1) # Decode Predicted labels
Y_test = np.argmax(y_test, 1) # Decode labels

mat = confusion_matrix(Y_test, Y_pred) # Confusion matrix

# Plot Confusion matrix
sns.heatmap(mat.T, square=True, annot=True, cbar=False, cmap=plt.cm.Blues)
plt.xlabel('Predicted Values')
plt.ylabel('True Values');
plt.show();

# **Prediction and Submition**

**Prediction validation results**

In [ ]:
y_pred = model.predict(X_test)
X_test__ = X_test.reshape(X_test.shape[0], 28, 28)

fig, axis = plt.subplots(4, 4, figsize=(12, 14))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_test__[i], cmap='binary')
    ax.set(title = f"Real Number is {y_test[i].argmax()}\nPredict Number is {y_pred[i].argmax()}");


**Prediciting the Outputs**

In [ ]:
pred = model.predict_classes(test_x, verbose=1)

In [ ]:
sub['Label'] = pred
sub.to_csv("CNN_keras_sub.csv", index=False)
sub.head()

We have got 99.5 % accuracy using Convolution Neural network

Please upvote my work if it could help! Thank you!